In [2]:
%pip install pandas numpy scipy matplotlib

  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl (40.9 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta

In [ ]:
class AdvancedMarketAnalyzer:
    def __init__(self, timeframe, risk_free_rate=0.02, dividend_yield=0.0):
        self.timeframe = timeframe
        self.rf = risk_free_rate
        self.dy = dividend_yield
        self.option_chain = None

    # Enhanced data processing with tick data support
    def resample_data(self, data, tick_data=None):
        data = data.copy()
        data['datetime'] = pd.to_datetime(data['datetime'])
        data.set_index('datetime', inplace=True)
        
        # Resample with volume profile
        resampled = data.resample(self.timeframe).agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()
        
        # Incorporate tick data if available
        if tick_data is not None:
            tick_data['datetime'] = pd.to_datetime(tick_data['datetime'])
            tick_data.set_index('datetime', inplace=True)
            resampled = resampled.merge(
                tick_data.resample(self.timeframe).agg({
                    'bid': 'last',
                    'ask': 'last',
                    'bid_size': 'sum',
                    'ask_size': 'sum'
                }), 
                left_index=True, right_index=True
            )
        
        return resampled

    def calculate_volume_profile(self, data, num_bins=25, lookback=200):
        """Calculate volume profile from price data"""
        if len(data) < lookback:
            return None
            
        recent_data = data.iloc[-lookback:]
        price_range = recent_data['high'].max() - recent_data['low'].min()
        
        # Pre-calculate bins
        bins = np.linspace(recent_data['low'].min(), recent_data['high'].max(), num_bins + 1)
        
        # Use numpy histogram for faster calculation
        volume_profile, _ = np.histogram(recent_data['close'], 
                                       bins=bins, 
                                       weights=recent_data['volume'])
        
        # Return with bin centers for plotting
        bin_centers = 0.5 * (bins[1:] + bins[:-1])
        return pd.Series(volume_profile, index=bin_centers)
    

    
    # Black-Scholes Greeks calculation
    def calculate_greeks(self, S, K, T, iv, option_type='call'):
        T = T / 365.0  # Convert days to years
        d1 = (np.log(S / K) + (self.rf - self.dy + 0.5 * iv**2) * T) / (iv * np.sqrt(T))
        d2 = d1 - iv * np.sqrt(T)
        
        if option_type == 'call':
            delta = norm.cdf(d1)
            gamma = norm.pdf(d1) / (S * iv * np.sqrt(T))
            theta = (-(S * norm.pdf(d1) * iv) / (2 * np.sqrt(T)) 
                     - self.rf * K * np.exp(-self.rf * T) * norm.cdf(d2)) / 365
            vega = S * norm.pdf(d1) * np.sqrt(T) / 100
        else:
            delta = -norm.cdf(-d1)
            gamma = norm.pdf(d1) / (S * iv * np.sqrt(T))
            theta = (-(S * norm.pdf(d1) * iv) / (2 * np.sqrt(T)) 
                     + self.rf * K * np.exp(-self.rf * T) * norm.cdf(-d2)) / 365
            vega = S * norm.pdf(d1) * np.sqrt(T) / 100
            
        return {
            'delta': delta,
            'gamma': gamma,
            'theta': theta,
            'vega': vega
        }

    # Enhanced indicators with volatility surface
    def calculate_indicators(self, data, iv_data=None):
        close = data['close']
        high = data['high']
        low = data['low']
        volume = data['volume']
        
        # Price-based indicators
        ma20 = close.rolling(20).mean()
        ma50 = close.rolling(50).mean()
        ma200 = close.rolling(200).mean()
        atr = self._calculate_atr(high, low, close)
        
        # Volatility indicators
        if iv_data is not None:
            iv_surface = {
                'atm': iv_data['atm'].iloc[-1],
                'skew': iv_data['25d_put'] - iv_data['25d_call'],
                'term_structure': iv_data['60d'] - iv_data['30d']
            }
        else:
            iv_surface = None
            
        # Volume profile
        vp = self.calculate_volume_profile(data)
        
        return {
            'ma20': ma20,
            'ma50': ma50,
            'ma200': ma200,
            'atr': atr,
            'iv_surface': iv_surface,
            'volume_profile': vp,
            'vwap': (data['volume'] * data['close']).cumsum() / data['volume'].cumsum()
        }

    # Vertical spread strategy engine
    def vertical_spread_strategy(self, data, indicators, option_chain):
        current_price = data['close'].iloc[-1]
        iv = indicators['iv_surface']['atm'] if indicators['iv_surface'] else 0.2
        atr = indicators['atr'].iloc[-1]
        days_to_expiry = self._days_to_expiry(option_chain['expiry'].iloc[0])
        
        # Strategy matrix
        strategies = {
            'bull_call': {
                'type': 'call',
                'direction': 'debit',
                'strikes': [
                    current_price - 0.5*atr,
                    current_price + 0.5*atr
                ],
                'conditions': (
                    indicators['ma20'] > indicators['ma50'] and
                    iv < 0.3 and
                    current_price > indicators['vwap'].iloc[-1]
                )
            },
            'bear_put': {
                'type': 'put',
                'direction': 'debit',
                'strikes': [
                    current_price + 0.5*atr,
                    current_price - 0.5*atr
                ],
                'conditions': (
                    indicators['ma20'] < indicators['ma50'] and
                    iv > 0.25 and
                    current_price < indicators['vwap'].iloc[-1]
                )
            },
            'bull_put': {
                'type': 'put',
                'direction': 'credit',
                'strikes': [
                    current_price + atr,
                    current_price
                ],
                'conditions': (
                    indicators['ma20'] > indicators['ma200'] and
                    iv > 0.35 and
                    data['volume'].iloc[-1] > data['volume'].rolling(20).mean().iloc[-1]
                )
            },
            'bear_call': {
                'type': 'call',
                'direction': 'credit',
                'strikes': [
                    current_price,
                    current_price + atr
                ],
                'conditions': (
                    indicators['ma20'] < indicators['ma200'] and
                    iv < 0.4 and
                    data['volume'].iloc[-1] < data['volume'].rolling(20).mean().iloc[-1]
                )
            }
        }
        
        # Select best strategy
        selected = None
        for name, strat in strategies.items():
            if strat['conditions']:
                selected = strat
                break
                
        if selected:
            # Calculate optimal strikes from option chain
            strikes = self._find_optimal_strikes(
                option_chain,
                selected['type'],
                selected['strikes'],
                days_to_expiry,
                iv
            )
            
            # Calculate risk parameters
            risk_params = self._calculate_risk_parameters(
                strikes,
                selected['type'],
                selected['direction'],
                days_to_expiry,
                current_price,
                iv
            )
            
            return {
                'strategy': name,
                'strikes': strikes,
                'risk_params': risk_params,
                'greeks': self._calculate_spread_greeks(strikes, selected, days_to_expiry, iv)
            }
        
        return None

    # Automated trade execution logic
    def execute_vertical_spread(self, strategy, account_size, risk_per_trade=0.01):
        max_loss = abs(strategy['risk_params']['max_loss'])
        position_size = (account_size * risk_per_trade) / max_loss
        position_size = int(position_size // 100) * 100  # Round to contract lots
        
        # Generate order legs
        legs = []
        if 'call' in strategy['strategy']:
            legs.append({
                'type': 'call',
                'strike': strategy['strikes'][0],
                'action': 'sell' if 'bear' in strategy['strategy'] else 'buy',
                'quantity': position_size
            })
            legs.append({
                'type': 'call',
                'strike': strategy['strikes'][1],
                'action': 'buy' if 'bear' in strategy['strategy'] else 'sell',
                'quantity': position_size
            })
        else:
            legs.append({
                'type': 'put',
                'strike': strategy['strikes'][0],
                'action': 'sell' if 'bull' in strategy['strategy'] else 'buy',
                'quantity': position_size
            })
            legs.append({
                'type': 'put',
                'strike': strategy['strikes'][1],
                'action': 'buy' if 'bull' in strategy['strategy'] else 'sell',
                'quantity': position_size
            })
            
        return {
            'orders': legs,
            'risk_parameters': {
                'stop_loss': strategy['risk_params']['breakeven'],
                'profit_target': strategy['risk_params']['max_profit'] * 0.8,
                'time_decay_threshold': strategy['greeks']['theta'] * 0.5
            }
        }

    # Helper methods
    def _calculate_atr(self, high, low, close, period=14):
        tr = np.maximum(high - low, 
                       np.abs(high - close.shift(1)),
                       np.abs(low - close.shift(1)))
        return tr.rolling(period).mean()
        
    def _days_to_expiry(self, expiry_date):
        return (expiry_date - datetime.now()).days
        
    def _find_optimal_strikes(self, chain, option_type, target_strikes, days_to_expiry, iv):
        # Filter options by type and expiry
        filtered = chain[
            (chain['type'] == option_type) & 
            (chain['days_to_expiry'].between(days_to_expiry-3, days_to_expiry+3))
        ]
        
        # Find nearest strikes with liquidity
        strikes = filtered[
            (filtered['strike'].between(target_strikes[0], target_strikes[1])) &
            (filtered['volume'] > 100) &
            (filtered['open_interest'] > 500)
        ]['strike'].values
        
        return [strikes[0], strikes[-1]] if len(strikes) >=2 else None
        
    def _calculate_risk_parameters(self, strikes, option_type, direction, days_to_expiry, price, iv):
        # Calculate spread parameters
        if direction == 'debit':
            cost = (self._get_option_price(strikes[0], option_type, days_to_expiry, iv, price) -
                    self._get_option_price(strikes[1], option_type, days_to_expiry, iv, price))
            max_profit = strikes[1] - strikes[0] - cost
            max_loss = cost
            breakeven = strikes[0] + cost if option_type == 'call' else strikes[0] - cost
        else:
            credit = (self._get_option_price(strikes[1], option_type, days_to_expiry, iv, price) -
                     self._get_option_price(strikes[0], option_type, days_to_expiry, iv, price))
            max_profit = credit
            max_loss = strikes[1] - strikes[0] - credit
            breakeven = strikes[0] + credit if option_type == 'call' else strikes[0] - credit
            
        return {
            'max_profit': max_profit,
            'max_loss': max_loss,
            'breakeven': breakeven,
            'risk_reward': max_profit / abs(max_loss)
        }
        
    def _get_option_price(self, strike, option_type, days_to_expiry, iv, spot):
        # Simple Black-Scholes implementation
        T = days_to_expiry / 365
        d1 = (np.log(spot/strike) + (self.rf + 0.5*iv**2)*T) / (iv*np.sqrt(T))
        d2 = d1 - iv*np.sqrt(T)
        
        if option_type == 'call':
            price = spot*norm.cdf(d1) - strike*np.exp(-self.rf*T)*norm.cdf(d2)
        else:
            price = strike*np.exp(-self.rf*T)*norm.cdf(-d2) - spot*norm.cdf(-d1)
            
        return price

    def _calculate_spread_greeks(self, strikes, strategy, days_to_expiry, iv):
        # Calculate net Greeks for the spread
        long_greeks = self.calculate_greeks(
            S=strikes[0], K=strikes[0], 
            T=days_to_expiry, iv=iv,
            option_type=strategy['type']
        )
        short_greeks = self.calculate_greeks(
            S=strikes[1], K=strikes[1],
            T=days_to_expiry, iv=iv,
            option_type=strategy['type']
        )
        
        # Net Greeks based on spread direction
        if strategy['direction'] == 'debit':
            return {
                'delta': long_greeks['delta'] - short_greeks['delta'],
                'gamma': long_greeks['gamma'] - short_greeks['gamma'],
                'theta': long_greeks['theta'] - short_greeks['theta'],
                'vega': long_greeks['vega'] - short_greeks['vega']
            }
        else:
            return {
                'delta': short_greeks['delta'] - long_greeks['delta'],
                'gamma': short_greeks['gamma'] - long_greeks['gamma'],
                'theta': short_greeks['theta'] - long_greeks['theta'],
                'vega': short_greeks['vega'] - long_greeks['vega']
            }


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta

class AdvancedMarketAnalyzer:
    def __init__(self, timeframe, risk_free_rate=0.02, dividend_yield=0.0):
        self.timeframe = timeframe
        self.rf = risk_free_rate
        self.dy = dividend_yield
        self.option_chain = None

    # Enhanced data processing with tick data support
    def resample_data(self, data, tick_data=None):
        data = data.copy()
        data['datetime'] = pd.to_datetime(data['datetime'])
        data.set_index('datetime', inplace=True)
        
        # Resample with volume profile
        resampled = data.resample(self.timeframe).agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()
        
        # Incorporate tick data if available
        if tick_data is not None:
            tick_data['datetime'] = pd.to_datetime(tick_data['datetime'])
            tick_data.set_index('datetime', inplace=True)
            resampled = resampled.merge(
                tick_data.resample(self.timeframe).agg({
                    'bid': 'last',
                    'ask': 'last',
                    'bid_size': 'sum',
                    'ask_size': 'sum'
                }), 
                left_index=True, right_index=True
            )
        
        return resampled

    # Black-Scholes Greeks calculation
    def calculate_greeks(self, S, K, T, iv, option_type='call'):
        T = T / 365.0  # Convert days to years
        d1 = (np.log(S / K) + (self.rf - self.dy + 0.5 * iv**2) * T) / (iv * np.sqrt(T))
        d2 = d1 - iv * np.sqrt(T)
        
        if option_type == 'call':
            delta = norm.cdf(d1)
            gamma = norm.pdf(d1) / (S * iv * np.sqrt(T))
            theta = (-(S * norm.pdf(d1) * iv) / (2 * np.sqrt(T)) 
                     - self.rf * K * np.exp(-self.rf * T) * norm.cdf(d2)) / 365
            vega = S * norm.pdf(d1) * np.sqrt(T) / 100
        else:
            delta = -norm.cdf(-d1)
            gamma = norm.pdf(d1) / (S * iv * np.sqrt(T))
            theta = (-(S * norm.pdf(d1) * iv) / (2 * np.sqrt(T)) 
                     + self.rf * K * np.exp(-self.rf * T) * norm.cdf(-d2)) / 365
            vega = S * norm.pdf(d1) * np.sqrt(T) / 100
            
        return {
            'delta': delta,
            'gamma': gamma,
            'theta': theta,
            'vega': vega
        }

    # Enhanced indicators with volatility surface
    def calculate_indicators(self, data, iv_data=None):
        close = data['close']
        high = data['high']
        low = data['low']
        volume = data['volume']
        
        # Price-based indicators
        ma20 = close.rolling(20).mean()
        ma50 = close.rolling(50).mean()
        ma200 = close.rolling(200).mean()
        atr = self._calculate_atr(high, low, close)
        
        # Volatility indicators
        if iv_data is not None:
            iv_surface = {
                'atm': iv_data['atm'].iloc[-1],
                'skew': iv_data['25d_put'] - iv_data['25d_call'],
                'term_structure': iv_data['60d'] - iv_data['30d']
            }
        else:
            iv_surface = None
            
        # Volume profile
        vp = self.calculate_volume_profile(data)
        
        return {
            'ma20': ma20,
            'ma50': ma50,
            'ma200': ma200,
            'atr': atr,
            'iv_surface': iv_surface,
            'volume_profile': vp,
            'vwap': (data['volume'] * data['close']).cumsum() / data['volume'].cumsum()
        }

    # Vertical spread strategy engine
    def vertical_spread_strategy(self, data, indicators, option_chain):
        current_price = data['close'].iloc[-1]
        iv = indicators['iv_surface']['atm'] if indicators['iv_surface'] else 0.2
        atr = indicators['atr'].iloc[-1]
        days_to_expiry = self._days_to_expiry(option_chain['expiry'].iloc[0])
        
        # Strategy matrix
        strategies = {
            'bull_call': {
                'type': 'call',
                'direction': 'debit',
                'strikes': [
                    current_price - 0.5*atr,
                    current_price + 0.5*atr
                ],
                'conditions': (
                    indicators['ma20'] > indicators['ma50'] and
                    iv < 0.3 and
                    current_price > indicators['vwap'].iloc[-1]
                )
            },
            'bear_put': {
                'type': 'put',
                'direction': 'debit',
                'strikes': [
                    current_price + 0.5*atr,
                    current_price - 0.5*atr
                ],
                'conditions': (
                    indicators['ma20'] < indicators['ma50'] and
                    iv > 0.25 and
                    current_price < indicators['vwap'].iloc[-1]
                )
            },
            'bull_put': {
                'type': 'put',
                'direction': 'credit',
                'strikes': [
                    current_price + atr,
                    current_price
                ],
                'conditions': (
                    indicators['ma20'] > indicators['ma200'] and
                    iv > 0.35 and
                    data['volume'].iloc[-1] > data['volume'].rolling(20).mean().iloc[-1]
                )
            },
            'bear_call': {
                'type': 'call',
                'direction': 'credit',
                'strikes': [
                    current_price,
                    current_price + atr
                ],
                'conditions': (
                    indicators['ma20'] < indicators['ma200'] and
                    iv < 0.4 and
                    data['volume'].iloc[-1] < data['volume'].rolling(20).mean().iloc[-1]
                )
            }
        }
        
        # Select best strategy
        selected = None
        for name, strat in strategies.items():
            if strat['conditions']:
                selected = strat
                break
                
        if selected:
            # Calculate optimal strikes from option chain
            strikes = self._find_optimal_strikes(
                option_chain,
                selected['type'],
                selected['strikes'],
                days_to_expiry,
                iv
            )
            
            # Calculate risk parameters
            risk_params = self._calculate_risk_parameters(
                strikes,
                selected['type'],
                selected['direction'],
                days_to_expiry,
                current_price,
                iv
            )
            
            return {
                'strategy': name,
                'strikes': strikes,
                'risk_params': risk_params,
                'greeks': self._calculate_spread_greeks(strikes, selected, days_to_expiry, iv)
            }
        
        return None

    # Automated trade execution logic
    def execute_vertical_spread(self, strategy, account_size, risk_per_trade=0.01):
        max_loss = abs(strategy['risk_params']['max_loss'])
        position_size = (account_size * risk_per_trade) / max_loss
        position_size = int(position_size // 100) * 100  # Round to contract lots
        
        # Generate order legs
        legs = []
        if 'call' in strategy['strategy']:
            legs.append({
                'type': 'call',
                'strike': strategy['strikes'][0],
                'action': 'sell' if 'bear' in strategy['strategy'] else 'buy',
                'quantity': position_size
            })
            legs.append({
                'type': 'call',
                'strike': strategy['strikes'][1],
                'action': 'buy' if 'bear' in strategy['strategy'] else 'sell',
                'quantity': position_size
            })
        else:
            legs.append({
                'type': 'put',
                'strike': strategy['strikes'][0],
                'action': 'sell' if 'bull' in strategy['strategy'] else 'buy',
                'quantity': position_size
            })
            legs.append({
                'type': 'put',
                'strike': strategy['strikes'][1],
                'action': 'buy' if 'bull' in strategy['strategy'] else 'sell',
                'quantity': position_size
            })
            
        return {
            'orders': legs,
            'risk_parameters': {
                'stop_loss': strategy['risk_params']['breakeven'],
                'profit_target': strategy['risk_params']['max_profit'] * 0.8,
                'time_decay_threshold': strategy['greeks']['theta'] * 0.5
            }
        }

    # Helper methods
    def _calculate_atr(self, high, low, close, period=14):
        tr = np.maximum(high - low, 
                       np.abs(high - close.shift(1)),
                       np.abs(low - close.shift(1)))
        return tr.rolling(period).mean()
        
    def _days_to_expiry(self, expiry_date):
        return (expiry_date - datetime.now()).days
        
    def _find_optimal_strikes(self, chain, option_type, target_strikes, days_to_expiry, iv):
        # Filter options by type and expiry
        filtered = chain[
            (chain['type'] == option_type) & 
            (chain['days_to_expiry'].between(days_to_expiry-3, days_to_expiry+3))
        ]
        
        # Find nearest strikes with liquidity
        strikes = filtered[
            (filtered['strike'].between(target_strikes[0], target_strikes[1])) &
            (filtered['volume'] > 100) &
            (filtered['open_interest'] > 500)
        ]['strike'].values
        
        return [strikes[0], strikes[-1]] if len(strikes) >=2 else None
        
    def _calculate_risk_parameters(self, strikes, option_type, direction, days_to_expiry, price, iv):
        # Calculate spread parameters
        if direction == 'debit':
            cost = (self._get_option_price(strikes[0], option_type, days_to_expiry, iv, price) -
                    self._get_option_price(strikes[1], option_type, days_to_expiry, iv, price))
            max_profit = strikes[1] - strikes[0] - cost
            max_loss = cost
            breakeven = strikes[0] + cost if option_type == 'call' else strikes[0] - cost
        else:
            credit = (self._get_option_price(strikes[1], option_type, days_to_expiry, iv, price) -
                     self._get_option_price(strikes[0], option_type, days_to_expiry, iv, price))
            max_profit = credit
            max_loss = strikes[1] - strikes[0] - credit
            breakeven = strikes[0] + credit if option_type == 'call' else strikes[0] - credit
            
        return {
            'max_profit': max_profit,
            'max_loss': max_loss,
            'breakeven': breakeven,
            'risk_reward': max_profit / abs(max_loss)
        }
        
    def _get_option_price(self, strike, option_type, days_to_expiry, iv, spot):
        # Simple Black-Scholes implementation
        T = days_to_expiry / 365
        d1 = (np.log(spot/strike) + (self.rf + 0.5*iv**2)*T) / (iv*np.sqrt(T))
        d2 = d1 - iv*np.sqrt(T)
        
        if option_type == 'call':
            price = spot*norm.cdf(d1) - strike*np.exp(-self.rf*T)*norm.cdf(d2)
        else:
            price = strike*np.exp(-self.rf*T)*norm.cdf(-d2) - spot*norm.cdf(-d1)
            
        return price

    def _calculate_spread_greeks(self, strikes, strategy, days_to_expiry, iv):
        # Calculate net Greeks for the spread
        long_greeks = self.calculate_greeks(
            S=strikes[0], K=strikes[0], 
            T=days_to_expiry, iv=iv,
            option_type=strategy['type']
        )
        short_greeks = self.calculate_greeks(
            S=strikes[1], K=strikes[1],
            T=days_to_expiry, iv=iv,
            option_type=strategy['type']
        )
        
        # Net Greeks based on spread direction
        if strategy['direction'] == 'debit':
            return {
                'delta': long_greeks['delta'] - short_greeks['delta'],
                'gamma': long_greeks['gamma'] - short_greeks['gamma'],
                'theta': long_greeks['theta'] - short_greeks['theta'],
                'vega': long_greeks['vega'] - short_greeks['vega']
            }
        else:
            return {
                'delta': short_greeks['delta'] - long_greeks['delta'],
                'gamma': short_greeks['gamma'] - long_greeks['gamma'],
                'theta': short_greeks['theta'] - long_greeks['theta'],
                'vega': short_greeks['vega'] - long_greeks['vega']
            }

# Example usage with live trading integration
if __name__ == "__main__":
    # Initialize analyzer with 15-minute timeframe
    analyzer = AdvancedMarketAnalyzer('15T')
    
    # Load sample data (replace with real-time feed)
    historical_data = pd.read_csv('market_data.csv')
    option_chain = pd.read_csv('options_chain.csv')
    
    # Resample data with tick information
    resampled_data = analyzer.resample_data(historical_data)
    
    # Calculate indicators with volatility surface
    indicators = analyzer.calculate_indicators(resampled_data)
    
    # Generate vertical spread strategy
    strategy = analyzer.vertical_spread_strategy(resampled_data, indicators, option_chain)
    
    if strategy:
        # Execute with $100k account size, 1% risk per trade
        trade_plan = analyzer.execute_vertical_spread(strategy, 100000)
        print("Generated Trading Plan:")
        print(f"Strategy: {strategy['strategy'].upper()}")
        print(f"Strikes: {strategy['strikes']}")
        print(f"Position Size: {trade_plan['orders'][0]['quantity']} contracts")
        print(f"Max Profit: ${strategy['risk_params']['max_profit']*100:.2f} per contract")
        print(f"Max Loss: ${strategy['risk_params']['max_loss']*100:.2f} per contract")
        print(f"Greek Exposure:", strategy['greeks'])
    else:
        print("No valid vertical spread opportunity detected")